# Picture classification SparkML


You have the MNIST dataset 

**Features :** 784 columns : Data type Number (Pixel Values in range 0, 255)   
**Label :** 1 column : Datatype Number (0, 9)  

Your task is to :

* Do some feature preprocessing

* Split the dataset into train and validation :
    * Train 70%
    * Test 30%
    * Use Random seed = 1

* Use any classifier you want (Naive Bayes, Decision Trees, Random Forest, GB Tree, NN e.t.c)

* Achieve test accuracy > 0.85 (>85%)
    * Use MulticlassClassificationEvaluator
    
* Write short summary about why you used the proposed model
  * Future improvements

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Init Spark Session

In [3]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

sc = SparkContext('local','example')  # if using locally
sql_sc = SQLContext(sc)



# Read Data

In [4]:
dataset = sql_sc.read.format("com.databricks.spark.csv")\
    .option("header", "true")\
    .option("inferschema", "true")\
    .option("mode", "DROPMALFORMED")\
    .load("dataset-w4.csv")


In [5]:
dataset.limit(5).toPandas()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
columns = dataset.columns
columns.remove('label')
features = columns
target = 'label'


# Transform Data

In [7]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, StandardScaler, Binarizer

In [7]:
dataset.printSchema()

root
 |-- label: integer (nullable = true)
 |-- pixel0: integer (nullable = true)
 |-- pixel1: integer (nullable = true)
 |-- pixel2: integer (nullable = true)
 |-- pixel3: integer (nullable = true)
 |-- pixel4: integer (nullable = true)
 |-- pixel5: integer (nullable = true)
 |-- pixel6: integer (nullable = true)
 |-- pixel7: integer (nullable = true)
 |-- pixel8: integer (nullable = true)
 |-- pixel9: integer (nullable = true)
 |-- pixel10: integer (nullable = true)
 |-- pixel11: integer (nullable = true)
 |-- pixel12: integer (nullable = true)
 |-- pixel13: integer (nullable = true)
 |-- pixel14: integer (nullable = true)
 |-- pixel15: integer (nullable = true)
 |-- pixel16: integer (nullable = true)
 |-- pixel17: integer (nullable = true)
 |-- pixel18: integer (nullable = true)
 |-- pixel19: integer (nullable = true)
 |-- pixel20: integer (nullable = true)
 |-- pixel21: integer (nullable = true)
 |-- pixel22: integer (nullable = true)
 |-- pixel23: integer (nullable = true)
 |-- pi

In [64]:
#Note: since all data is in integer, no need to label them into float or something.

In [8]:
assembler = VectorAssembler()\
.setInputCols(features)\
.setOutputCol("features")


transformed_data = assembler.transform(dataset)
inputDF = transformed_data.select('label', 'features')

In [9]:
#inputDF
inputDF.toPandas().head()

,label,features
0,1,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,1,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,4,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [10]:
inputDF.select(inputDF.label).distinct().toPandas()

,label
0,1
1,6
2,3
3,5
4,9
5,4
6,8
7,7
8,2
9,0


# Perform random train/test split

In [10]:
train, test =  inputDF.randomSplit([0.7, 0.3], seed = 1)

In [11]:
train.limit(3).toPandas()

,label,features
0,0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [12]:
test.limit(3).toPandas()

,label,features
0,0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


# Define classifier

In [13]:
from pyspark.ml.classification import *
from pyspark.ml.classification import RandomForestClassifier

In [14]:
# create random forest model

rfClassifier = RandomForestClassifier(labelCol='label', featuresCol='features', numTrees=100)

# Fit model

In [15]:
rfModel = rfClassifier.fit(train)

In [16]:
rfModel.featureImportances

SparseVector(784, {66: 0.0001, 67: 0.0001, 70: 0.0002, 71: 0.0003, 72: 0.0003, 97: 0.0012, 98: 0.0022, 99: 0.0028, 100: 0.0014, 101: 0.0031, 102: 0.0008, 103: 0.0008, 122: 0.0001, 123: 0.0007, 124: 0.0004, 125: 0.001, 126: 0.0021, 127: 0.0021, 128: 0.0021, 129: 0.0007, 130: 0.0, 131: 0.0002, 148: 0.0001, 149: 0.0011, 150: 0.0013, 151: 0.0028, 152: 0.0041, 153: 0.006, 154: 0.0109, 155: 0.0048, 156: 0.0085, 157: 0.0032, 158: 0.0026, 159: 0.0004, 160: 0.0003, 161: 0.0004, 162: 0.0, 175: 0.0002, 176: 0.0006, 177: 0.0009, 178: 0.0017, 179: 0.004, 180: 0.0011, 181: 0.0012, 182: 0.0042, 183: 0.0045, 184: 0.0028, 185: 0.001, 186: 0.0011, 187: 0.0008, 188: 0.0002, 189: 0.0005, 190: 0.0009, 191: 0.0001, 193: 0.0001, 202: 0.0001, 203: 0.0005, 204: 0.0003, 205: 0.0014, 206: 0.0009, 207: 0.0022, 208: 0.0008, 209: 0.0045, 210: 0.005, 211: 0.003, 212: 0.0041, 213: 0.0019, 214: 0.0021, 215: 0.003, 216: 0.0007, 217: 0.0005, 218: 0.0009, 219: 0.0005, 220: 0.0004, 230: 0.0005, 231: 0.0003, 232: 0.001, 23

In [17]:
rfModel.toDebugString

'RandomForestClassificationModel (uid=RandomForestClassifier_5978c92369d3) with 100 trees\n  Tree 0 (weight 1.0):\n    If (feature 567 <= 0.5)\n     If (feature 323 <= 177.5)\n      If (feature 127 <= 1.5)\n       If (feature 237 <= 3.5)\n        If (feature 152 <= 11.5)\n         Predict: 4.0\n        Else (feature 152 > 11.5)\n         Predict: 8.0\n       Else (feature 237 > 3.5)\n        If (feature 270 <= 0.5)\n         Predict: 5.0\n        Else (feature 270 > 0.5)\n         Predict: 7.0\n      Else (feature 127 > 1.5)\n       If (feature 270 <= 1.5)\n        If (feature 658 <= 0.5)\n         Predict: 6.0\n        Else (feature 658 > 0.5)\n         Predict: 5.0\n       Else (feature 270 > 1.5)\n        If (feature 324 <= 10.5)\n         Predict: 0.0\n        Else (feature 324 > 10.5)\n         Predict: 8.0\n     Else (feature 323 > 177.5)\n      If (feature 550 <= 0.5)\n       If (feature 436 <= 170.5)\n        If (feature 178 <= 0.5)\n         Predict: 1.0\n        Else (feature

In [18]:
# toido make prediction
predictions = rfModel.transform(test)

predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|    0|(784,[68,69,97,15...|[53.2610606860283...|[0.53261060686028...|       0.0|
|    0|(784,[92,93,96,97...|[11.5882326564168...|[0.11588232656416...|       6.0|
|    0|(784,[93,94,95,96...|[71.2984365581068...|[0.71298436558106...|       0.0|
|    0|(784,[93,97,98,99...|[43.0194807545899...|[0.43019480754589...|       0.0|
|    0|(784,[94,95,96,97...|[67.5768029329031...|[0.67576802932903...|       0.0|
|    0|(784,[94,95,96,97...|[35.0579572162206...|[0.35057957216220...|       0.0|
|    0|(784,[94,95,96,97...|[54.1884067089659...|[0.54188406708965...|       0.0|
|    0|(784,[94,95,99,10...|[62.4225438764484...|[0.62422543876448...|       0.0|
|    0|(784,[95,96,97,98...|[78.1947573831823...|[0.78194757383182...|       0.0|
|    0|(784,[95,

# Evaluate model on test data 

In [19]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# make accuracy
evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName="accuracy")

In [20]:
accuracy = evaluator.evaluate(predictions)

print("Accuracy", accuracy)
print("Teset error", (1-accuracy))

Accuracy 0.8548888357638059
Teset error 0.14511116423619408


### Decision Tree

In [21]:
decisionTree = DecisionTreeClassifier(labelCol="label", featuresCol='features', maxDepth=15, maxBins=16)

In [22]:
dtModel = decisionTree.fit(train)

In [24]:
dtModel.numNodes

4025

In [25]:
dtModel.depth

15

In [23]:
predictions = dtModel.transform(test)

predictions.show(10)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|    0|(784,[68,69,97,15...|[0.0,0.0,0.0,0.0,...|[0.0,0.0,0.0,0.0,...|       7.0|
|    0|(784,[92,93,96,97...|[1.0,0.0,0.0,0.0,...|[1.0,0.0,0.0,0.0,...|       0.0|
|    0|(784,[93,94,95,96...|[1812.0,0.0,0.0,0...|[0.99505766062602...|       0.0|
|    0|(784,[93,97,98,99...|[1812.0,0.0,0.0,0...|[0.99505766062602...|       0.0|
|    0|(784,[94,95,96,97...|[271.0,0.0,1.0,0....|[0.98188405797101...|       0.0|
|    0|(784,[94,95,96,97...|[0.0,0.0,0.0,0.0,...|[0.0,0.0,0.0,0.0,...|       5.0|
|    0|(784,[94,95,96,97...|[271.0,0.0,1.0,0....|[0.98188405797101...|       0.0|
|    0|(784,[94,95,99,10...|[1812.0,0.0,0.0,0...|[0.99505766062602...|       0.0|
|    0|(784,[95,96,97,98...|[1812.0,0.0,0.0,0...|[0.99505766062602...|       0.0|
|    0|(784,[95,

In [24]:
# todo print prediction
predictions.select('label', 'probability', 'prediction').show()

+-----+--------------------+----------+
|label|         probability|prediction|
+-----+--------------------+----------+
|    0|[0.0,0.0,0.0,0.0,...|       7.0|
|    0|[1.0,0.0,0.0,0.0,...|       0.0|
|    0|[0.99505766062602...|       0.0|
|    0|[0.99505766062602...|       0.0|
|    0|[0.98188405797101...|       0.0|
|    0|[0.0,0.0,0.0,0.0,...|       5.0|
|    0|[0.98188405797101...|       0.0|
|    0|[0.99505766062602...|       0.0|
|    0|[0.99505766062602...|       0.0|
|    0|[0.0,0.0,0.0,0.0,...|       6.0|
|    0|[0.99505766062602...|       0.0|
|    0|[0.99505766062602...|       0.0|
|    0|[0.99505766062602...|       0.0|
|    0|[0.99505766062602...|       0.0|
|    0|[0.98188405797101...|       0.0|
|    0|[0.99505766062602...|       0.0|
|    0|[0.98188405797101...|       0.0|
|    0|[0.99505766062602...|       0.0|
|    0|[0.99505766062602...|       0.0|
|    0|[0.99505766062602...|       0.0|
+-----+--------------------+----------+
only showing top 20 rows



In [25]:
#Model evaluation
evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName="accuracy")

accuracy = evaluator.evaluate(predictions)

print("Accuracy", accuracy)
print("Test error", (1-accuracy))

Accuracy 0.8561638377559965
Test error 0.14383616224400353


## Normalizing dataset

In [26]:
inputDF.limit(5).toPandas()

,label,features
0,1,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,1,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,4,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [27]:
from pyspark.ml.feature import MinMaxScaler
assembler = VectorAssembler()\
.setInputCols(features)\
.setOutputCol("features")

transformed = assembler.transform(dataset)

scaler = MinMaxScaler(inputCol="features",\
         outputCol="scaledFeatures")

scalerModel =  scaler.fit(transformed.select("features"))
scaledData = scalerModel.transform(transformed)

In [28]:
scaledData.limit(5).toPandas()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,features,scaledFeatures
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, ..."
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, ..."
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, ..."
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, ..."
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, ..."


In [29]:
inputDF_scaller = scaledData.select('label', 'scaledFeatures')

In [30]:
inputDF_scaller.limit(5).toPandas()

,label,scaledFeatures
0,1,"[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, ..."
1,0,"[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, ..."
2,1,"[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, ..."
3,4,"[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, ..."
4,0,"[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, ..."


In [31]:
# Perform random train/test split
train, test =  inputDF_scaller.randomSplit([0.7, 0.3], seed = 1)

In [32]:
# create random forest model
rfClassifier_scal = RandomForestClassifier(labelCol='label', featuresCol='scaledFeatures', numTrees=100)

In [33]:
rfModel_scal = rfClassifier_scal.fit(train)

In [34]:
# make prediction
predictions = rfModel_scal.transform(test)

predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|      scaledFeatures|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|    0|[0.5,0.5,0.5,0.5,...|[63.4136140426914...|[0.63413614042691...|       0.0|
|    0|[0.5,0.5,0.5,0.5,...|[52.0719619295936...|[0.52071961929593...|       0.0|
|    0|[0.5,0.5,0.5,0.5,...|[3.76233474184347...|[0.03762334741843...|       4.0|
|    0|[0.5,0.5,0.5,0.5,...|[69.6320172494247...|[0.69632017249424...|       0.0|
|    0|[0.5,0.5,0.5,0.5,...|[47.8447354790709...|[0.47844735479070...|       0.0|
|    0|[0.5,0.5,0.5,0.5,...|[36.8663998430735...|[0.36866399843073...|       0.0|
|    0|[0.5,0.5,0.5,0.5,...|[49.8722342983655...|[0.49872234298365...|       0.0|
|    0|[0.5,0.5,0.5,0.5,...|[76.1458739315571...|[0.76145873931557...|       0.0|
|    0|[0.5,0.5,0.5,0.5,...|[11.6240981720239...|[0.11624098172023...|       6.0|
|    0|[0.5,0.5,

In [35]:
# make accuracy
evaluator_scal = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName="accuracy")

In [36]:
accuracy = evaluator_scal.evaluate(predictions)

print("Accuracy", accuracy)
print("Teset error", (1-accuracy))

Accuracy 0.8500278906685792
Teset error 0.1499721093314208


In [102]:
#NOTE: Min/Max scaller doesn't give improved results

## Neural Networsk

In [37]:
assembler = VectorAssembler()\
.setInputCols(features)\
.setOutputCol("features")


transformed_data = assembler.transform(dataset)
inputDF = transformed_data.select('label', 'features')

In [38]:
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol='features', outputCol='scaled_features', 
                       withStd=True, withMean=True)

scalerTransformer = scaler.fit(transformed_data)
transformed_data = scalerTransformer.transform(transformed_data)


transformed_data.limit(5).toPandas()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,features,scaled_features
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [39]:
transformed_data_short = transformed_data.select('label', 'scaled_features')
transformed_data_short.limit(3).toPandas()

,label,scaled_features
0,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [40]:
trainingData, testData = transformed_data_short.randomSplit([0.7, 0.3], seed=1)

In [41]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# todo trian neural network
layers = [32, 16, 10] 

In [42]:
perceptron = MultilayerPerceptronClassifier(maxIter=150, layers=layers, featuresCol="scaled_features", labelCol="label")

In [43]:
# training
model=perceptron.fit(trainingData)

In [44]:
model

MultilayerPerceptronClassifier_48b7a07a014d

In [45]:
# prediction
result = model.transform(testData)

In [46]:
evaluator = MulticlassClassificationEvaluator(metricName="accuracy", labelCol="label", 
                                              predictionCol='prediction')

In [52]:
evaluator.evaluate(result, None)

Py4JJavaError: An error occurred while calling o758.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 158.0 failed 1 times, most recent failure: Lost task 0.0 in stage 158.0 (TID 156, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$1: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:636)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNe

In [53]:
evaluator.evaluate(result, {})

Py4JJavaError: An error occurred while calling o758.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 160.0 failed 1 times, most recent failure: Lost task 0.0 in stage 160.0 (TID 157, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$1: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:636)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNe

In [51]:
print("Test set error = " + str(1-evaluator.evaluate(result)))

Py4JJavaError: An error occurred while calling o758.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 156.0 failed 1 times, most recent failure: Lost task 0.0 in stage 156.0 (TID 155, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$1: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:636)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNe

# Summary